In [22]:
from ultralytics import YOLO
import cv2

# Load YOLO models
vehicle_model = YOLO('yolov8s.pt')  # COCO-pretrained model for vehicles
helmet_model = YOLO('helmet-detector.pt')  # Custom model for drivers and helmets

# Video input
video_path = 'C:/Users/JINAY DOSHI/Desktop/finalproject/testvideos/test3.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Unable to open video file.")
    exit()

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS) or 30
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Output video
output_video_path = 'C:/Users/JINAY DOSHI/Desktop/finalproject/outputvideos/output6.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Tracking data
object_ids = {}  # Maps object centers to unique IDs
next_id = 1  # Counter for assigning unique IDs

# Define COCO vehicle classes to detect (car, bus, truck)
VEHICLE_CLASSES = {2: "Car", 5: "Bus", 7: "Truck"}  # COCO class IDs for vehicles

def get_closest_id(center, existing_centers, max_distance=50):
    """ Match the new detection center with existing centers within a threshold distance. """
    for existing_id, existing_center in existing_centers.items():
        distance = ((center[0] - existing_center[0]) ** 2 + (center[1] - existing_center[1]) ** 2) ** 0.5
        if distance <= max_distance:
            return existing_id
    return None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    current_centers = {}

    # Detect vehicles
    vehicle_results = vehicle_model.predict(frame, stream=True, conf=0.25)
    for result in vehicle_results:
        boxes = result.boxes
        for box in boxes:
            class_id = int(box.cls[0])
            if class_id not in VEHICLE_CLASSES:
                continue

            x1, y1, x2, y2 = map(int, box.xyxy[0])
            center = ((x1 + x2) // 2, (y1 + y2) // 2)

            unique_id = get_closest_id(center, object_ids)
            if unique_id is None:
                unique_id = next_id
                next_id += 1

            object_ids[unique_id] = center
            current_centers[unique_id] = center

            # Draw bounding box and label
            label = f"ID: {unique_id} {VEHICLE_CLASSES[class_id]}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 1)  # Yellow for vehicles
            cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)

    # Detect drivers and helmets
    helmet_results = helmet_model.predict(frame, stream=True)
    for helmet_result in helmet_results:
        boxes = helmet_result.boxes
        for hbox in boxes:
            hx1, hy1, hx2, hy2 = map(int, hbox.xyxy[0])
            class_id = int(hbox.cls[0])

            if class_id == 1:  # Driver
                center = ((hx1 + hx2) // 2, (hy1 + hy2) // 2)

                unique_id = get_closest_id(center, object_ids)
                if unique_id is None:
                    unique_id = next_id
                    next_id += 1

                object_ids[unique_id] = center
                current_centers[unique_id] = center

                label = f"ID: {unique_id} Driver"
                cv2.rectangle(frame, (hx1, hy1), (hx2, hy2), (0, 255, 255), 1)  # Yellow for drivers
                cv2.putText(frame, label, (hx1, hy1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)

            elif class_id in [2, 3]:  # Helmet or No Helmet
                color = (0, 255, 0) if class_id == 2 else (0, 0, 255)
                label = "Helmet" if class_id == 2 else "No Helmet"
                cv2.putText(frame, label, (hx1, hy1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)
                cv2.rectangle(frame, (hx1, hy1), (hx2, hy2), color, 1)

    object_ids = current_centers
    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()
print(f"✅ Processing complete. Video saved at: {output_video_path}")



0: 384x640 1 bicycle, 5 cars, 1 suitcase, 156.4ms
Speed: 5.0ms preprocess, 156.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 driver, 238.5ms
Speed: 4.0ms preprocess, 238.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bicycle, 5 cars, 1 suitcase, 148.7ms
Speed: 3.0ms preprocess, 148.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 driver, 136.3ms
Speed: 2.0ms preprocess, 136.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bicycle, 5 cars, 1 suitcase, 122.5ms
Speed: 2.2ms preprocess, 122.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 driver, 122.0ms
Speed: 2.0ms preprocess, 122.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bicycle, 6 cars, 1 suitcase, 116.7ms
Speed: 2.0ms preprocess, 116.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 driver

# Speed & Helmet detection 


In [33]:
from ultralytics import YOLO
import cv2
import numpy as np

# Load YOLO models
vehicle_model = YOLO('yolov8s.pt')  # COCO-pretrained model for vehicles
helmet_model = YOLO('helmet-detector.pt')  # Custom model for drivers and helmets

# Video input
video_path = 'C:/Users/JINAY DOSHI/Desktop/finalproject/testvideos/test3.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Unable to open video file.")
    exit()

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS) or 30
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Output video
output_video_path = 'C:/Users/JINAY DOSHI/Desktop/finalproject/outputvideos/output10.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Tracking data
object_ids = {}  # Maps object centers to unique IDs
previous_positions = {}  # Stores the last known positions of objects
previous_frames = {}  # Stores the frame numbers for speed calculation
max_speed_dict = {}  # Stores the maximum speed detected for each object
next_id = 1  # Counter for assigning unique IDs

# Define COCO vehicle classes to detect (car, bus, truck)
VEHICLE_CLASSES = {2: "Car", 5: "Bus", 7: "Truck"}  # COCO class IDs for vehicles

# Calibration factor (adjust based on real-world measurement)
PIXEL_TO_METER = 0.05  # Conversion factor from pixel distance to meters

def get_closest_id(center, existing_centers, max_distance=50):
    """ Match the new detection center with existing centers within a threshold distance. """
    for existing_id, existing_center in existing_centers.items():
        distance = ((center[0] - existing_center[0]) ** 2 + (center[1] - existing_center[1]) ** 2) ** 0.5
        if distance <= max_distance:
            return existing_id
    return None

while cap.isOpened():
    ret, frame = cap.read()
    current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))

    if not ret:
        break

    current_centers = {}

    # Detect vehicles
    vehicle_results = vehicle_model.predict(frame, stream=True, conf=0.25)
    for result in vehicle_results:
        boxes = result.boxes
        for box in boxes:
            class_id = int(box.cls[0])
            if class_id not in VEHICLE_CLASSES:
                continue

            x1, y1, x2, y2 = map(int, box.xyxy[0])
            center = ((x1 + x2) // 2, (y1 + y2) // 2)

            unique_id = get_closest_id(center, object_ids)
            if unique_id is None:
                unique_id = next_id
                next_id += 1

            object_ids[unique_id] = center
            current_centers[unique_id] = center

            # Speed Calculation
            if unique_id in previous_positions:
                prev_center = previous_positions[unique_id]
                prev_frame = previous_frames[unique_id]
                pixel_distance = np.linalg.norm(np.array(center) - np.array(prev_center))
                time_elapsed = (current_frame - prev_frame) / fps
                speed_kmph = (pixel_distance * PIXEL_TO_METER) / time_elapsed * 3.6 if time_elapsed > 0 else 0
            else:
                speed_kmph = 0

            # Store the maximum speed detected
            max_speed_dict[unique_id] = max(max_speed_dict.get(unique_id, 0), speed_kmph)

            # Update previous positions and frames
            previous_positions[unique_id] = center
            previous_frames[unique_id] = current_frame

            # Draw bounding box and label
            label = f"ID: {unique_id} {VEHICLE_CLASSES[class_id]} {int(max_speed_dict[unique_id])} km/h"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 1)  # Yellow for vehicles
            cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)

    # Detect drivers and helmets
    helmet_results = helmet_model.predict(frame, stream=True)
    for helmet_result in helmet_results:
        boxes = helmet_result.boxes
        for hbox in boxes:
            hx1, hy1, hx2, hy2 = map(int, hbox.xyxy[0])
            class_id = int(hbox.cls[0])

            if class_id == 1:  # Driver
                center = ((hx1 + hx2) // 2, (hy1 + hy2) // 2)

                unique_id = get_closest_id(center, object_ids)
                if unique_id is None:
                    unique_id = next_id
                    next_id += 1

                object_ids[unique_id] = center
                current_centers[unique_id] = center

                # Speed Calculation
                if unique_id in previous_positions:
                    prev_center = previous_positions[unique_id]
                    prev_frame = previous_frames[unique_id]
                    pixel_distance = np.linalg.norm(np.array(center) - np.array(prev_center))
                    time_elapsed = (current_frame - prev_frame) / fps
                    speed_kmph = (pixel_distance * PIXEL_TO_METER) / time_elapsed * 3.6 if time_elapsed > 0 else 0
                else:
                    speed_kmph = 0

                max_speed_dict[unique_id] = max(max_speed_dict.get(unique_id, 0), speed_kmph)

                previous_positions[unique_id] = center
                previous_frames[unique_id] = current_frame

                label = f"ID: {unique_id} Driver {int(max_speed_dict[unique_id])} km/h"
                cv2.rectangle(frame, (hx1, hy1), (hx2, hy2), (0, 255, 255), 1)  # Yellow for drivers
                cv2.putText(frame, label, (hx1, hy1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)

            elif class_id in [2, 3]:  # Helmet or No Helmet
                color = (0, 255, 0) if class_id == 2 else (0, 0, 255)
                label = "Helmet" if class_id == 2 else "No Helmet"
                cv2.putText(frame, label, (hx1, hy1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)
                cv2.rectangle(frame, (hx1, hy1), (hx2, hy2), color, 1)

    object_ids = current_centers
    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()
print(f"✅ Processing complete. Video saved at: {output_video_path}")



0: 384x640 1 bicycle, 5 cars, 1 suitcase, 140.3ms
Speed: 6.0ms preprocess, 140.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 driver, 263.5ms
Speed: 2.7ms preprocess, 263.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bicycle, 5 cars, 1 suitcase, 253.0ms
Speed: 3.3ms preprocess, 253.0ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 driver, 322.3ms
Speed: 3.0ms preprocess, 322.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bicycle, 5 cars, 1 suitcase, 254.9ms
Speed: 4.5ms preprocess, 254.9ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 driver, 241.2ms
Speed: 4.0ms preprocess, 241.2ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bicycle, 6 cars, 1 suitcase, 229.2ms
Speed: 2.0ms preprocess, 229.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 driver

# stored image


In [73]:
from ultralytics import YOLO
import cv2
import numpy as np
import os

# Load YOLO models
vehicle_model = YOLO('yolov8s.pt')  # COCO-pretrained model for vehicles
helmet_model = YOLO('helmet-detector.pt')  # Custom model for drivers and helmets

# Video input
video_path = 'C:/Users/JINAY DOSHI/Desktop/finalproject/testvideos/test3.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Unable to open video file.")
    exit()

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS) or 30
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Output video
output_video_path = 'C:/Users/JINAY DOSHI/Desktop/finalproject/outputvideos/output18.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Directory to save captured images
os.makedirs("captured_images", exist_ok=True)

# Tracking data
object_ids = {}
previous_positions = {}
previous_frames = {}
max_speed_dict = {}
next_id = 1

# Annotator line properties (25% from the bottom of the frame)
annotator_line_y = int(frame_height * 0.75)
crossed_line = set()  # Track objects that have already crossed the line

# Calibration factor
PIXEL_TO_METER = 0.05  # Conversion factor from pixel distance to meters

# Store driver bounding boxes for No Helmet association
driver_bboxes = {}

def get_closest_id(center, existing_centers, max_distance=50):
    """ Match the new detection center with existing centers within a threshold distance. """
    for existing_id, existing_center in existing_centers.items():
        distance = np.linalg.norm(np.array(center) - np.array(existing_center))
        if distance <= max_distance:
            return existing_id
    return None

while cap.isOpened():
    ret, frame = cap.read()
    current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))

    if not ret:
        break

    current_centers = {}

    # Draw invisible annotator line
    cv2.line(frame, (0, annotator_line_y), (frame_width, annotator_line_y), (255, 255, 255), 1)

    # Step 1: Detect vehicles
    vehicle_results = vehicle_model.predict(frame, stream=True, conf=0.25)
    for result in vehicle_results:
        boxes = result.boxes
        for box in boxes:
            class_id = int(box.cls[0])
            if class_id not in {2, 5, 7}:  # Only process Car, Bus, Truck
                continue

            x1, y1, x2, y2 = map(int, box.xyxy[0])
            center = ((x1 + x2) // 2, (y1 + y2) // 2)

            unique_id = get_closest_id(center, object_ids)
            if unique_id is None:
                unique_id = next_id
                next_id += 1

            object_ids[unique_id] = center
            current_centers[unique_id] = center

            # Speed Calculation
            if unique_id in previous_positions:
                prev_center = previous_positions[unique_id]
                prev_frame = previous_frames[unique_id]
                pixel_distance = np.linalg.norm(np.array(center) - np.array(prev_center))
                time_elapsed = (current_frame - prev_frame) / fps
                speed_kmph = (pixel_distance * PIXEL_TO_METER) / time_elapsed * 3.6 if time_elapsed > 0 else 0
            else:
                speed_kmph = 0

            max_speed_dict[unique_id] = max(max_speed_dict.get(unique_id, 0), speed_kmph)

            previous_positions[unique_id] = center
            previous_frames[unique_id] = current_frame

            label = f"ID: {unique_id} Vehicle {int(max_speed_dict[unique_id])} km/h"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 1)  # Yellow for vehicles
            cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)

            # Save image if speed > 55 km/h and crosses the annotator line
            if center[1] >= annotator_line_y and unique_id not in crossed_line and max_speed_dict[unique_id] > 55:
                crossed_line.add(unique_id)
                image_filename = f"captured_images/vehicle_{unique_id}_frame_{current_frame}.jpg"
                cropped_image = frame[max(0, y1):min(frame_height, y2), max(0, x1):min(frame_width, x2)]
                cv2.imwrite(image_filename, cropped_image)
                print(f"Saved image: {image_filename}")

    # Step 2: Detect drivers and helmets
    helmet_results = helmet_model.predict(frame, stream=True)
    for helmet_result in helmet_results:
        boxes = helmet_result.boxes
        for hbox in boxes:
            hx1, hy1, hx2, hy2 = map(int, hbox.xyxy[0])
            class_id = int(hbox.cls[0])

            if class_id == 1:  # Driver
                center = ((hx1 + hx2) // 2, (hy1 + hy2) // 2)

                unique_id = get_closest_id(center, object_ids)
                if unique_id is None:
                    unique_id = next_id
                    next_id += 1

                object_ids[unique_id] = center
                current_centers[unique_id] = center

                # Save driver bounding box for No Helmet association
                driver_bboxes[unique_id] = (hx1, hy1, hx2, hy2)

                # Speed Calculation
                if unique_id in previous_positions:
                    prev_center = previous_positions[unique_id]
                    prev_frame = previous_frames[unique_id]
                    pixel_distance = np.linalg.norm(np.array(center) - np.array(prev_center))
                    time_elapsed = (current_frame - prev_frame) / fps
                    speed_kmph = (pixel_distance * PIXEL_TO_METER) / time_elapsed * 3.6 if time_elapsed > 0 else 0
                else:
                    speed_kmph = 0

                max_speed_dict[unique_id] = max(max_speed_dict.get(unique_id, 0), speed_kmph)

                previous_positions[unique_id] = center
                previous_frames[unique_id] = current_frame

                label = f"ID: {unique_id} Driver {int(max_speed_dict[unique_id])} km/h"
                cv2.rectangle(frame, (hx1, hy1), (hx2, hy2), (0, 255, 255), 1)  # Yellow for drivers
                cv2.putText(frame, label, (hx1, hy1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)

                # Save image if speed > 45 km/h and crosses the annotator line
                if center[1] >= annotator_line_y and unique_id not in crossed_line and max_speed_dict[unique_id] > 45:
                    crossed_line.add(unique_id)
                    image_filename = f"captured_images/driver_{unique_id}_frame_{current_frame}.jpg"
                    cropped_image = frame[max(0, hy1):min(frame_height, hy2), max(0, hx1):min(frame_width, hx2)]
                    cv2.imwrite(image_filename, cropped_image)
                    print(f"Saved image: {image_filename}")

            elif class_id == 3:  # No Helmet
                label = "No Helmet"
                cv2.rectangle(frame, (hx1, hy1), (hx2, hy2), (0, 0, 255), 1)
                cv2.putText(frame, label, (hx1, hy1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)

                # Save full driver image if No Helmet detected and crosses the annotator line
                for driver_id, (dx1, dy1, dx2, dy2) in driver_bboxes.items():
                    if hx1 >= dx1 and hy1 >= dy1 and hx2 <= dx2 and hy2 <= dy2:  # Ensure No Helmet is within driver box
                        if center[1] >= annotator_line_y and driver_id not in crossed_line:
                            crossed_line.add(driver_id)
                            image_filename = f"captured_images/no_helmet_{driver_id}_frame_{current_frame}.jpg"
                            cropped_image = frame[max(0, dy1):min(frame_height, dy2), max(0, dx1):min(frame_width, dx2)]
                            cv2.imwrite(image_filename, cropped_image)
                            print(f"Saved image: {image_filename}")

    object_ids = current_centers
    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()
print(f"✅ Processing complete. Video saved at: {output_video_path}")



0: 384x640 1 bicycle, 6 cars, 1 suitcase, 192.7ms
Speed: 5.0ms preprocess, 192.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 driver, 305.1ms
Speed: 3.7ms preprocess, 305.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bicycle, 6 cars, 1 suitcase, 274.3ms
Speed: 4.5ms preprocess, 274.3ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 driver, 204.2ms
Speed: 3.0ms preprocess, 204.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bicycle, 5 cars, 1 suitcase, 261.9ms
Speed: 3.5ms preprocess, 261.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 191.7ms
Speed: 3.0ms preprocess, 191.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bicycle, 6 cars, 1 suitcase, 156.4ms
Speed: 3.1ms preprocess, 156.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1

KeyboardInterrupt: 

# car&driver detection + speed + helmet + license plate

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np

# Load YOLO models
vehicle_model = YOLO('yolov8s.pt')  # COCO-pretrained model for vehicles
helmet_model = YOLO('helmet-detector.pt')  # Custom model for drivers and helmets
license_plate_model = YOLO('license_plate_detector.pt')  # Model for license plate detection

# Video input
video_path = 'C:/Users/JINAY DOSHI/Desktop/finalproject/testvideos/test3.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Unable to open video file.")
    exit()

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS) or 30
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Output video
output_video_path = 'C:/Users/JINAY DOSHI/Desktop/finalproject/outputvideos/output22.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Tracking data
object_ids = {}  # Maps object centers to unique IDs
previous_positions = {}  # Stores the last known positions of objects
previous_frames = {}  # Stores the frame numbers for speed calculation
max_speed_dict = {}  # Stores the maximum speed detected for each object
printlist={}
helmet_status_dict = {} 
next_id = 1  # Counter for assigning unique IDs

# Define COCO vehicle classes to detect (car, bus, truck)
VEHICLE_CLASSES = {2: "Car", 5: "Bus", 7: "Truck"}  # COCO class IDs for vehicles

# Calibration factor (adjust based on real-world measurement)
PIXEL_TO_METER = 0.05  # Conversion factor from pixel distance to meters

def get_closest_id(center, existing_centers, max_distance=50):
    """ Match the new detection center with existing centers within a threshold distance. """
    for existing_id, existing_center in existing_centers.items():
        distance = ((center[0] - existing_center[0]) ** 2 + (center[1] - existing_center[1]) ** 2) ** 0.5
        if distance <= max_distance:
            return existing_id
    return None

while cap.isOpened():
    ret, frame = cap.read()
    current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))

    if not ret:
        break

    current_centers = {}

    # Detect vehicles
    vehicle_results = vehicle_model.predict(frame, stream=True, conf=0.25)
    for result in vehicle_results:
        boxes = result.boxes
        for box in boxes:
            class_id = int(box.cls[0])
            if class_id not in VEHICLE_CLASSES:
                continue

            x1, y1, x2, y2 = map(int, box.xyxy[0])
            center = ((x1 + x2) // 2, (y1 + y2) // 2)

            unique_id = get_closest_id(center, object_ids)
            if unique_id is None:
                unique_id = next_id
                next_id += 1
                
                # print("-----------------------------------------------------------------------------------------------------------")
                # print(unique_id)
                
                if unique_id not in printlist.keys():
                    printlist.update({unique_id:[VEHICLE_CLASSES[class_id],-1,'UNKNOWN']})
                
            object_ids[unique_id] = center
            current_centers[unique_id] = center

            # Speed Calculation
            if unique_id in previous_positions:
                prev_center = previous_positions[unique_id]
                prev_frame = previous_frames[unique_id]
                pixel_distance = np.linalg.norm(np.array(center) - np.array(prev_center))
                time_elapsed = (current_frame - prev_frame) / fps
                speed_kmph = (pixel_distance * PIXEL_TO_METER) / time_elapsed * 3.6 if time_elapsed > 0 else 0
            else:
                speed_kmph = 0

            # Store the maximum speed detected
            max_speed_dict[unique_id] = max(max_speed_dict.get(unique_id, 0), speed_kmph)
            printlist[unique_id][1]=int(max_speed_dict[unique_id])

            # Update previous positions and frames
            previous_positions[unique_id] = center
            previous_frames[unique_id] = current_frame

            # Draw bounding box and label
            label = f"ID: {unique_id} {VEHICLE_CLASSES[class_id]} {int(max_speed_dict[unique_id])} km/h"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 1)  # Yellow for vehicles
            cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)

    # Detect drivers and helmets
    helmet_results = helmet_model.predict(frame, stream=True)
    for helmet_result in helmet_results:
        boxes = helmet_result.boxes
        for hbox in boxes:
            hx1, hy1, hx2, hy2 = map(int, hbox.xyxy[0])
            class_id = int(hbox.cls[0])

            if class_id == 1:  # Driver
                center = ((hx1 + hx2) // 2, (hy1 + hy2) // 2)

                unique_id = get_closest_id(center, object_ids)
                if unique_id is None:
                    unique_id = next_id
                    next_id += 1

                    if unique_id not in printlist.keys():
                        printlist.update({unique_id:['driver',-1,'UNKNOWN']})
                
                object_ids[unique_id] = center
                current_centers[unique_id] = center

                # Speed Calculation
                if unique_id in previous_positions:
                    prev_center = previous_positions[unique_id]
                    prev_frame = previous_frames[unique_id]
                    pixel_distance = np.linalg.norm(np.array(center) - np.array(prev_center))
                    time_elapsed = (current_frame - prev_frame) / fps
                    speed_kmph = (pixel_distance * PIXEL_TO_METER) / time_elapsed * 3.6 if time_elapsed > 0 else 0
                else:
                    speed_kmph = 0

                max_speed_dict[unique_id] = max(max_speed_dict.get(unique_id, 0), speed_kmph)
                printlist[unique_id][1]=int(max_speed_dict[unique_id])

                previous_positions[unique_id] = center
                previous_frames[unique_id] = current_frame

                # Default label to "Unknown" if no helmet status is detected yet
                helmet_status = helmet_status_dict.get(unique_id, "Unknown")
                label = f"ID: {unique_id} Driver {int(max_speed_dict[unique_id])} km/h {helmet_status}"
                cv2.rectangle(frame, (hx1, hy1), (hx2, hy2), (0, 255, 255), 1)  # Yellow for drivers
                cv2.putText(frame, label, (hx1, hy1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)

            elif class_id in [2, 3]:  # Helmet or No Helmet
                color = (0, 255, 0) if class_id == 2 else (0, 0, 255)
                helmet_status = "Helmet" if class_id == 2 else "No Helmet"

                # Associate helmet status with the closest driver
                center = ((hx1 + hx2) // 2, (hy1 + hy2) // 2)
                unique_id = get_closest_id(center, object_ids)
                if unique_id is not None:
                    helmet_status_dict[unique_id] = helmet_status
                    printlist[unique_id][2]=helmet_status
                
                cv2.putText(frame, helmet_status, (hx1, hy1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)
                cv2.rectangle(frame, (hx1, hy1), (hx2, hy2), color, 1)


    # Detect license plates
    license_plate_results = license_plate_model.predict(frame, stream=True)
    for lp_result in license_plate_results:
        boxes = lp_result.boxes
        for lp_box in boxes:
            lp_x1, lp_y1, lp_x2, lp_y2 = map(int, lp_box.xyxy[0])
            label = "License Plate"
            cv2.rectangle(frame, (lp_x1, lp_y1), (lp_x2, lp_y2), (255, 0, 0), 1)  # Blue for license plates
            cv2.putText(frame, label, (lp_x1, lp_y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 0, 0), 1)

    object_ids = current_centers
    out.write(frame)
    cv2.ijmshow

cap.release()
out.release()
cv2.destroyAllWindows()
print(f"✅ Processing complete. Video saved at: {output_video_path}")



0: 384x640 1 bicycle, 5 cars, 1 suitcase, 136.4ms
Speed: 2.5ms preprocess, 136.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 driver, 196.0ms
Speed: 2.0ms preprocess, 196.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 89.1ms
Speed: 3.4ms preprocess, 89.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bicycle, 5 cars, 1 suitcase, 188.7ms
Speed: 2.0ms preprocess, 188.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 driver, 157.9ms
Speed: 3.0ms preprocess, 157.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 74.6ms
Speed: 2.0ms preprocess, 74.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bicycle, 5 cars, 1 suitcase, 163.7ms
Speed: 4.5ms preprocess, 163.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 driver, 161.8ms
S

In [6]:
from ultralytics import YOLO
import cv2
import numpy as np

# Load YOLO models
vehicle_model = YOLO('yolov8s.pt')  # COCO-pretrained model for vehicles
helmet_model = YOLO('helmet-detector.pt')  # Custom model for drivers and helmets
license_plate_model = YOLO('license_plate_detector.pt')  # Model for license plate detection

# Video input
video_path = 'C:/Users/JINAY DOSHI/Desktop/finalproject/testvideos/highway.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Unable to open video file.")
    exit()

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS) or 30
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Output video
output_video_path = 'C:/Users/JINAY DOSHI/Desktop/finalproject/finaloutput/highway2.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Set processing start and end times
start_time_sec = 0  # Start at 20 seconds
end_time_sec = 50   # End at 30 seconds
start_frame = int(start_time_sec * fps)
end_frame = int(end_time_sec * fps)

# Seek to the start frame
cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

# Tracking data
object_ids = {}
previous_positions = {}
previous_frames = {}
max_speed_dict = {}
printlist = {}
helmet_status_dict = {}
next_id = 1

# Define COCO vehicle classes to detect (car, bus, truck)
VEHICLE_CLASSES = {2: "Car", 5: "Bus", 7: "Truck"}  # COCO class IDs for vehicles

# Calibration factor (adjust based on real-world measurement)
PIXEL_TO_METER = 0.035  # Conversion factor from pixel distance to meters

def get_closest_id(center, existing_centers, max_distance=50):
    """ Match the new detection center with existing centers within a threshold distance. """
    for existing_id, existing_center in existing_centers.items():
        distance = ((center[0] - existing_center[0]) ** 2 + (center[1] - existing_center[1]) ** 2) ** 0.5
        if distance <= max_distance:
            return existing_id
    return None

while cap.isOpened():
    current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
    
    # Stop processing when the end frame is reached
    if current_frame >= end_frame:
        break

    ret, frame = cap.read()

    if not ret:
        break

    current_centers = {}

    # (The rest of your YOLO detection and processing code goes here)
    # (Refer to your original code to detect vehicles, helmets, license plates, and calculate speeds)
 # Detect vehicles
    vehicle_results = vehicle_model.predict(frame, stream=True, conf=0.25)
    for result in vehicle_results:
        boxes = result.boxes
        for box in boxes:
            class_id = int(box.cls[0])
            if class_id not in VEHICLE_CLASSES:
                continue

            x1, y1, x2, y2 = map(int, box.xyxy[0])
            center = ((x1 + x2) // 2, (y1 + y2) // 2)

            unique_id = get_closest_id(center, object_ids)
            if unique_id is None:
                unique_id = next_id
                next_id += 1
                
                # print("-----------------------------------------------------------------------------------------------------------")
                # print(unique_id)
                
                if unique_id not in printlist.keys():
                    printlist.update({unique_id:[VEHICLE_CLASSES[class_id],-1,'UNKNOWN']})
                
            object_ids[unique_id] = center
            current_centers[unique_id] = center

            # Speed Calculation
            if unique_id in previous_positions:
                prev_center = previous_positions[unique_id]
                prev_frame = previous_frames[unique_id]
                pixel_distance = np.linalg.norm(np.array(center) - np.array(prev_center))
                time_elapsed = (current_frame - prev_frame) / fps
                speed_kmph = (pixel_distance * PIXEL_TO_METER) / time_elapsed * 3.6 if time_elapsed > 0 else 0
            else:
                speed_kmph = 0

            # Store the maximum speed detected
            max_speed_dict[unique_id] = max(max_speed_dict.get(unique_id, 0), speed_kmph)
            printlist[unique_id][1]=int(max_speed_dict[unique_id])

            # Update previous positions and frames
            previous_positions[unique_id] = center
            previous_frames[unique_id] = current_frame

            # Draw bounding box and label
            label = f"ID: {unique_id} {VEHICLE_CLASSES[class_id]} {int(max_speed_dict[unique_id])} km/h"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 1)  # Yellow for vehicles
            cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)

    # Detect drivers and helmets
    helmet_results = helmet_model.predict(frame, stream=True)
    for helmet_result in helmet_results:
        boxes = helmet_result.boxes
        for hbox in boxes:
            hx1, hy1, hx2, hy2 = map(int, hbox.xyxy[0])
            class_id = int(hbox.cls[0])

            if class_id == 1:  # Driver
                center = ((hx1 + hx2) // 2, (hy1 + hy2) // 2)

                unique_id = get_closest_id(center, object_ids)
                if unique_id is None:
                    unique_id = next_id
                    next_id += 1

                    if unique_id not in printlist.keys():
                        printlist.update({unique_id:['driver',-1,'UNKNOWN']})
                
                object_ids[unique_id] = center
                current_centers[unique_id] = center

                # Speed Calculation
                if unique_id in previous_positions:
                    prev_center = previous_positions[unique_id]
                    prev_frame = previous_frames[unique_id]
                    pixel_distance = np.linalg.norm(np.array(center) - np.array(prev_center))
                    time_elapsed = (current_frame - prev_frame) / fps
                    speed_kmph = (pixel_distance * PIXEL_TO_METER) / time_elapsed * 3.6 if time_elapsed > 0 else 0
                else:
                    speed_kmph = 0

                max_speed_dict[unique_id] = max(max_speed_dict.get(unique_id, 0), speed_kmph)
                printlist[unique_id][1]=int(max_speed_dict[unique_id])

                previous_positions[unique_id] = center
                previous_frames[unique_id] = current_frame

                # Default label to "Unknown" if no helmet status is detected yet
                helmet_status = helmet_status_dict.get(unique_id, "Unknown")
                label = f"ID: {unique_id} Driver {int(max_speed_dict[unique_id])} km/h {helmet_status}"
                cv2.rectangle(frame, (hx1, hy1), (hx2, hy2), (0, 255, 255), 1)  # Yellow for drivers
                cv2.putText(frame, label, (hx1, hy1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)

            elif class_id in [2, 3]:  # Helmet or No Helmet
                color = (0, 255, 0) if class_id == 2 else (0, 0, 255)
                helmet_status = "Helmet" if class_id == 2 else "No Helmet"

                # Associate helmet status with the closest driver
                center = ((hx1 + hx2) // 2, (hy1 + hy2) // 2)
                unique_id = get_closest_id(center, object_ids)
                if unique_id is not None:
                    helmet_status_dict[unique_id] = helmet_status
                    printlist[unique_id][2]=helmet_status
                
                cv2.putText(frame, helmet_status, (hx1, hy1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)
                cv2.rectangle(frame, (hx1, hy1), (hx2, hy2), color, 1)


    # Detect license plates
    license_plate_results = license_plate_model.predict(frame, stream=True)
    for lp_result in license_plate_results:
        boxes = lp_result.boxes
        for lp_box in boxes:
            lp_x1, lp_y1, lp_x2, lp_y2 = map(int, lp_box.xyxy[0])
            label = "License Plate"
            cv2.rectangle(frame, (lp_x1, lp_y1), (lp_x2, lp_y2), (255, 0, 0), 1)  # Blue for license plates
            cv2.putText(frame, label, (lp_x1, lp_y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 0, 0), 1)

    object_ids = current_centers
    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()
print(f"✅ Processing complete. Video saved at: {output_video_path}")



0: 384x640 9 cars, 3 trucks, 111.9ms
Speed: 2.6ms preprocess, 111.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 helmet, 129.2ms
Speed: 4.0ms preprocess, 129.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 48.6ms
Speed: 2.5ms preprocess, 48.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 4 trucks, 111.1ms
Speed: 3.0ms preprocess, 111.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 101.8ms
Speed: 2.0ms preprocess, 101.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 45.2ms
Speed: 2.3ms preprocess, 45.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 3 trucks, 104.5ms
Speed: 2.0ms preprocess, 104.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 helmet, 99.6ms
Speed: 2.0ms preprocess, 99.6ms i

In [8]:
import pymysql
from ultralytics import YOLO
import cv2
import numpy as np

# Database connection
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='',
    database='vehicle_data'  # Replace with your database name
)
cursor = connection.cursor()

# Create table if it doesn't exist
cursor.execute("""
    CREATE TABLE IF NOT EXISTS vehicle_records (
        id INT AUTO_INCREMENT PRIMARY KEY,
        unique_id INT,
        vehicle_type VARCHAR(50),
        max_speed FLOAT,
        helmet_status VARCHAR(50)
    )
""")

# Load YOLO models
vehicle_model = YOLO('yolov8s.pt')  # COCO-pretrained model for vehicles
helmet_model = YOLO('helmet-detector.pt')  # Custom model for drivers and helmets
license_plate_model = YOLO('best.pt')  # Model for license plate detection

# Video input
video_path = 'C:/Users/JINAY DOSHI/Desktop/finalproject/testvideos/test3.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Unable to open video file.")
    exit()

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS) or 30
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Set start and end times
start_time_sec = 0  # Start processing from 20 seconds
end_time_sec = 100    # Stop processing at 30 seconds
start_frame = int(start_time_sec * fps)
end_frame = int(end_time_sec * fps)

# Seek to the start frame
cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

# Output video
output_video_path = 'C:/Users/JINAY DOSHI/Desktop/finalproject/finaloutput/test333.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Tracking data
object_ids = {}
previous_positions = {}
previous_frames = {}
max_speed_dict = {}
printlist = {}
helmet_status_dict = {}
next_id = 1

# Define COCO vehicle classes to detect (car, bus, truck)
VEHICLE_CLASSES = {2: "Car", 5: "Bus", 7: "Truck"}

# Calibration factor
PIXEL_TO_METER = 0.01

def get_closest_id(center, existing_centers, max_distance=50):
    """ Match the new detection center with existing centers within a threshold distance. """
    for existing_id, existing_center in existing_centers.items():
        distance = ((center[0] - existing_center[0]) ** 2 + (center[1] - existing_center[1]) ** 2) ** 0.5
        if distance <= max_distance:
            return existing_id
    return None

while cap.isOpened():
    current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))

    # Stop processing when the end frame is reached
    if current_frame >= end_frame:
        break

    ret, frame = cap.read()

    if not ret:
        break

    current_centers = {}

    # Detect vehicles
    vehicle_results = vehicle_model.predict(frame, stream=True, conf=0.75)
    for result in vehicle_results:
        boxes = result.boxes
        for box in boxes:
            class_id = int(box.cls[0])
            if class_id not in VEHICLE_CLASSES:
                continue

            x1, y1, x2, y2 = map(int, box.xyxy[0])
            center = ((x1 + x2) // 2, (y1 + y2) // 2)

            unique_id = get_closest_id(center, object_ids)
            if unique_id is None:
                unique_id = next_id
                next_id += 1

                if unique_id not in printlist.keys():
                    printlist[unique_id] = [VEHICLE_CLASSES[class_id], -1, 'Unknown']

            object_ids[unique_id] = center
            current_centers[unique_id] = center

            # Speed Calculation
            if unique_id in previous_positions:
                prev_center = previous_positions[unique_id]
                prev_frame = previous_frames[unique_id]
                pixel_distance = np.linalg.norm(np.array(center) - np.array(prev_center))
                time_elapsed = (current_frame - prev_frame) / fps
                speed_kmph = (pixel_distance * PIXEL_TO_METER) / time_elapsed * 3.6 if time_elapsed > 0 else 0
            else:
                speed_kmph = 0

            max_speed_dict[unique_id] = max(max_speed_dict.get(unique_id, 0), speed_kmph)
            printlist[unique_id][1] = int(max_speed_dict[unique_id])

            previous_positions[unique_id] = center
            previous_frames[unique_id] = current_frame

            label = f"ID: {unique_id} {VEHICLE_CLASSES[class_id]} {int(max_speed_dict[unique_id])} km/h"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 1)
            cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)

    # Detect drivers and helmets
    helmet_results = helmet_model.predict(frame, stream=True)
    for helmet_result in helmet_results:
        boxes = helmet_result.boxes
        for hbox in boxes:
            hx1, hy1, hx2, hy2 = map(int, hbox.xyxy[0])
            class_id = int(hbox.cls[0])

            if class_id == 1:  # Driver
                center = ((hx1 + hx2) // 2, (hy1 + hy2) // 2)
                unique_id = get_closest_id(center, object_ids)
                if unique_id is None:
                    unique_id = next_id
                    next_id += 1
                    if unique_id not in printlist:
                        printlist[unique_id] = ['Driver', -1, 'Unknown']
                object_ids[unique_id] = center
                current_centers[unique_id] = center

                # Speed Calculation (same as before)
                if unique_id in previous_positions:
                    prev_center = previous_positions[unique_id]
                    prev_frame = previous_frames[unique_id]
                    pixel_distance = np.linalg.norm(np.array(center) - np.array(prev_center))
                    time_elapsed = (current_frame - prev_frame) / fps
                    speed_kmph = (pixel_distance * PIXEL_TO_METER) / time_elapsed * 3.6 if time_elapsed > 0 else 0
                else:
                    speed_kmph = 0

                max_speed_dict[unique_id] = max(max_speed_dict.get(unique_id, 0), speed_kmph)
                printlist[unique_id][1] = int(max_speed_dict[unique_id])

                previous_positions[unique_id] = center
                previous_frames[unique_id] = current_frame

                # # Ensure latest helmet status is used
                helmet_status = helmet_status_dict.get(unique_id, "Unknown")
                printlist[unique_id][2] = helmet_status  # Reflect in printlist

                # Display updated label
                label = f"ID: {unique_id} Driver {int(max_speed_dict[unique_id])} km/h {helmet_status}"
                cv2.rectangle(frame, (hx1, hy1), (hx2, hy2), (0, 255, 255), 1)
                cv2.putText(frame, label, (hx1, hy1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)

            elif class_id in [2, 3]:  # Helmet or No Helmet
                color = (0, 255, 0) if class_id == 2 else (0, 0, 255)
                helmet_status = "Helmet" if class_id == 2 else "No Helmet"

                # Associate the helmet status with the closest driver
                center = ((hx1 + hx2) // 2, (hy1 + hy2) // 2)
                unique_id = get_closest_id(center, object_ids)
                if unique_id is not None:
                    helmet_status_dict[unique_id] = helmet_status  # Update the status
                    printlist[unique_id][2] = helmet_status  # Reflect status in printlist

                # Draw helmet/no-helmet label
                cv2.putText(frame, helmet_status, (hx1, hy1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)
                cv2.rectangle(frame, (hx1, hy1), (hx2, hy2), color, 1)
                
                # # Ensure latest helmet status is used
                # helmet_status = helmet_status_dict.get(unique_id, "Unknown")
                # printlist[unique_id][2] = helmet_status  # Reflect in printlist

                # # Display updated label
                # label = f"ID: {unique_id} Driver {int(max_speed_dict[unique_id])} km/h {helmet_status}"
                # cv2.rectangle(frame, (hx1, hy1), (hx2, hy2), (0, 255, 255), 1)
                # cv2.putText(frame, label, (hx1, hy1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)

                
    # Detect license plates
    license_plate_results = license_plate_model.predict(frame, stream=True)
    for lp_result in license_plate_results:
        boxes = lp_result.boxes
        for lp_box in boxes:
            lp_x1, lp_y1, lp_x2, lp_y2 = map(int, lp_box.xyxy[0])
            label = "License Plate"
            cv2.rectangle(frame, (lp_x1, lp_y1), (lp_x2, lp_y2), (255, 0, 0), 1)  # Blue for license plates
            cv2.putText(frame, label, (lp_x1, lp_y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 0, 0), 1)

    object_ids = current_centers
    out.write(frame)

# Store data in the database
for unique_id, data in printlist.items():
    vehicle_type, max_speed, helmet_status = data
    cursor.execute("""
        INSERT INTO vehicle_records (unique_id, vehicle_type, max_speed, helmet_status)
        VALUES (%s, %s, %s, %s)
    """, (unique_id, vehicle_type, max_speed, helmet_status))
    connection.commit()

# Release resources
cap.release()
out.release()
cursor.close()
connection.close()
print(f"✅ Processing complete. Video saved at: {output_video_path}")



0: 384x640 5 cars, 158.5ms
Speed: 11.0ms preprocess, 158.5ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 driver, 123.1ms
Speed: 2.0ms preprocess, 123.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 102.8ms
Speed: 3.0ms preprocess, 102.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 171.5ms
Speed: 4.0ms preprocess, 171.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 driver, 126.1ms
Speed: 2.0ms preprocess, 126.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 66.7ms
Speed: 2.0ms preprocess, 66.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 131.8ms
Speed: 2.2ms preprocess, 131.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 driver, 126.4ms
Speed: 2.5ms preprocess, 126.4ms inference, 1.0ms postprocess per i